# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [31]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import numpy as np

In [32]:
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)

In [33]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [34]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [35]:
song_files = get_files('data/song_data')

In [36]:
song_files[:3]

['/home/workspace/data/song_data/A/A/A/TRAAADZ128F9348C2E.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json']

In [37]:
filepath = song_files[3]

In [38]:
df = pd.read_json(filepath,lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [39]:
df.year

0    1969
Name: year, dtype: int64

In [40]:
song_data = df.loc[0,['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()

song_data

['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [41]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [42]:
artist_data = df.loc[0,['artist_id' , 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values.tolist()
artist_data

['ARMJAGH1187FB546F3',
 'The Box Tops',
 'Memphis, TN',
 35.149679999999996,
 -90.048919999999995]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [43]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [44]:
log_files = get_files('data/log_data')

In [45]:
log_files

['/home/workspace/data/log_data/2018/11/2018-11-06-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-12-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-14-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-05-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-24-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-10-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-17-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-28-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-21-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-09-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-25-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-13-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-16-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-20-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-22-events.json',
 '/home/workspace/data/lo

In [46]:
filepath = log_files[0]

In [47]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Adler,M,0,Barrera,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540836e+12,248,None,200,1541470364796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
1,Gustavo Cerati,Logged In,Adler,M,1,Barrera,249.44281,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Uno Entre 1000,200,1541470383796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
2,Limp Bizkit,Logged In,Adler,M,2,Barrera,270.49751,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Behind Blue Eyes,200,1541470632796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
3,None,Logged In,Samuel,M,0,Gonzalez,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540493e+12,252,None,200,1541473967796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
4,Mikel Erentxun,Logged In,Samuel,M,1,Gonzalez,178.83383,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,252,Frases Mudas,200,1541474048796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [48]:
df = df.loc[df.page == 'NextSong', :]
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Gustavo Cerati,Logged In,Adler,M,1,Barrera,249.44281,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Uno Entre 1000,200,1541470383796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100


In [49]:
t = pd.to_datetime(df.ts, unit='ms')
t.head()

1   2018-11-06 02:13:03.796
2   2018-11-06 02:17:12.796
4   2018-11-06 03:14:08.796
5   2018-11-06 04:56:11.796
6   2018-11-06 04:56:38.796
Name: ts, dtype: datetime64[ns]

In [50]:
#list(df.ts)

In [51]:
time_data = {'start_time':list(df.ts), 'hour': list(t.dt.hour), 'day': (t.dt.day), 'week': list(t.dt.week), 'month': list(t.dt.month), 'year': list(t.dt.year),\
             'weekday':list(t.dt.weekday)}
#column_labels = ()


In [52]:
time_df = pd.DataFrame(time_data)
time_df.head()

,start_time,hour,day,week,month,year,weekday
1,1541470383796,2,6,45,11,2018,1
2,1541470632796,2,6,45,11,2018,1
4,1541474048796,3,6,45,11,2018,1
5,1541480171796,4,6,45,11,2018,1
6,1541480198796,4,6,45,11,2018,1


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [53]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [54]:
user_df = df.loc[:, ['userId','firstName','lastName', 'gender', 'level']]

In [55]:
user_df.head()

,userId,firstName,lastName,gender,level
1,100,Adler,Barrera,M,free
2,100,Adler,Barrera,M,free
4,61,Samuel,Gonzalez,M,free
5,55,Martin,Johnson,M,free
6,55,Martin,Johnson,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [56]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [57]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [58]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.